In [15]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from scipy.spatial import KDTree
from dataclasses import dataclass
from data_utils import LatticeDataProcessor
from caesar.logger.logger import setup_logger

logger = setup_logger(__name__)
data_processor = LatticeDataProcessor()


@dataclass
class parameters:
    traj_file_path = "./data/200k-trajectory.xyz"
    traj_output_dir = "./data/timesteps_200k"  # 添加输出目录
    step = 50000
    lattice_output_dir = "./data/lattice_slices_200k/Ti/"
    frame_path = "./data/timesteps_200k/frame_50000.npy"  # 要处理的帧数据路径
    lattice_constant = 3.9127  # 晶格常数
    tolerance = 1.9  # 偏移
    cubic_length = 20  # 立方相的y方向扩胞数
    cubic_output_dir = "./data/cubic_slices/"
    # born 有效电荷
    Z_A_Pb = np.array([3.74, 3.74, 3.45])
    Z_A_Sr = np.array([2.56, 2.56, 2.56])
    Z_Ti_Pb = np.array([6.17, 6.17, 5.21])
    Z_Ti_Sr = np.array([7.4, 7.4, 7.4])
    Z_O_Pb = np.array([-3.3, -3.3, -2.89])
    Z_O_Sr = np.array([-3.32, -3.32, -3.32])


def read_data(data_path: str):
    data = np.load(data_path)
    return data


def process_A_coord(raw: list):
    # 根据中心点坐标[0.5,0.5,0.5]处理A位原子, 共八个顶角, [0,0,0],[0,0,1],[0,1,0],[0,1,1],[1,0,0],[1,0,1],[1,1,0],[1,1,1]
    A_coord = [[0, 0, 0]] * 8
    # [0,0,0]
    A_coord[0] = raw[0] - parameters.lattice_constant / 2, \
        raw[1] - parameters.lattice_constant / 2, \
        raw[2] - parameters.lattice_constant / 2
    # [0,0,1]
    A_coord[1] = raw[0] - parameters.lattice_constant / 2, \
        raw[1] - parameters.lattice_constant / 2, \
        raw[2] + parameters.lattice_constant / 2
    # [0,1,0]
    A_coord[2] = raw[0] - parameters.lattice_constant / 2, \
        raw[1] + parameters.lattice_constant / 2, \
        raw[2] - parameters.lattice_constant / 2
    # [0,1,1]
    A_coord[3] = raw[0] - parameters.lattice_constant / 2, \
        raw[1] + parameters.lattice_constant / 2, \
        raw[2] + parameters.lattice_constant / 2
    # [1,0,0]
    A_coord[4] = raw[0] + parameters.lattice_constant / 2, \
        raw[1] - parameters.lattice_constant / 2, \
        raw[2] - parameters.lattice_constant / 2
    # [1,0,1]
    A_coord[5] = raw[0] + parameters.lattice_constant / 2, \
        raw[1] - parameters.lattice_constant / 2, \
        raw[2] + parameters.lattice_constant / 2
    # [1,1,0]
    A_coord[6] = raw[0] + parameters.lattice_constant / 2, \
        raw[1] + parameters.lattice_constant / 2, \
        raw[2] - parameters.lattice_constant / 2
    # [1,1,1]
    A_coord[7] = raw[0] + parameters.lattice_constant / 2, \
        raw[1] + parameters.lattice_constant / 2, \
        raw[2] + parameters.lattice_constant / 2
    # 将A_coord转换为numpy数组
    A_coord = np.array(A_coord)
    return A_coord


def process_O_coord(raw: list):
    # 根据中心点坐标[0.5,0.5,0.5]处理O位原子, 共六个面心, [0.5, 0.5, 0], [0.5, 0.5, 1], [0.5, 0, 0.5], [0.5, 1, 0.5], [0, 0.5, 0.5], [1, 0.5, 0.5]
    O_coord = [[0, 0, 0]] * 6
    # [0.5, 0.5, 0]
    O_coord[0] = raw[0], raw[1], raw[2] - parameters.lattice_constant / 2
    # [0.5, 0.5, 1]
    O_coord[1] = raw[0], raw[1], raw[2] + parameters.lattice_constant / 2
    # [0.5, 0, 0.5]
    O_coord[2] = raw[0], raw[1] - parameters.lattice_constant / 2, raw[2]
    # [0.5, 1, 0.5]
    O_coord[3] = raw[0], raw[1] + parameters.lattice_constant / 2, raw[2]
    # [0, 0.5, 0.5]
    O_coord[4] = raw[0] - parameters.lattice_constant / 2, raw[1], raw[2]
    # [1, 0.5, 0.5]
    O_coord[5] = raw[0] + parameters.lattice_constant / 2, raw[1], raw[2]
    # 将O_coord转换为numpy数组
    O_coord = np.array(O_coord)

    return O_coord


def cal_polarization(A_coord: np.ndarray, Ti_coord: np.ndarray, O_coord: np.ndarray, A_cubic: np.ndarray,
                     Ti_cubic: np.ndarray, O_cubic: np.ndarray, volume: float, atom_coord: np.ndarray, kdtree: KDTree):
    A_label = [int(i) for i in A_coord[:, 0]]
    label_unique = np.unique(A_label)

    if len(label_unique) == 2:
        A_offset = np.array(A_coord[:, [1, 2, 3]] - A_cubic)
        Ti_offset = np.array(Ti_coord[[1, 2, 3]] - Ti_cubic)
        O_offset = np.array(O_coord[:, [1, 2, 3]] - O_cubic)

        Ti_pol = Ti_offset*parameters.Z_Ti_Pb
        # 判断O原子的位置，这里没有完善，是STO占主导还是PTO
        O_pol = np.sum(O_offset * parameters.Z_O_Sr, axis=0) / 2

        A_pol = []
        for i in range(len(A_coord)):
            if A_coord[i][0] == 1:
                A_pol.append(A_offset[i]*parameters.Z_A_Pb)
            else:
                A_pol.append(A_offset[i]*parameters.Z_A_Sr)
        A_pol = np.sum(A_pol, axis=0) / 8
        pol = (A_pol + Ti_pol + O_pol) / volume
        # logger.info(f'pol: \n{pol}')
    elif len(label_unique) == 1:
        A_offset = np.array(A_coord[:, [1, 2, 3]] - A_cubic)
        Ti_offset = np.array(Ti_coord[[1, 2, 3]] - Ti_cubic)
        O_offset = np.array(O_coord[:, [1, 2, 3]] - O_cubic)
        if label_unique == 2:
            A_pol = np.sum(A_offset * parameters.Z_A_Sr, axis=0) / 8
            Ti_pol = Ti_offset * parameters.Z_Ti_Sr
            O_pol = np.sum(O_offset * parameters.Z_O_Sr, axis=0) / 2

            pol = (A_pol + Ti_pol + O_pol) / volume
            # logger.info(f'pol: \n{pol}')
        if label_unique == 1:
            A_pol = np.sum(A_offset * parameters.Z_A_Pb, axis=0) / 8
            Ti_pol = Ti_offset * parameters.Z_Ti_Pb
            O_pol = np.sum(O_offset * parameters.Z_O_Pb, axis=0) / 2

            pol = (A_pol + Ti_pol + O_pol) / volume
            # logger.info(f'pol: \n{pol}')

    else:
        logger.warning('A_label 中存在未知标签')
        raise ValueError('A_label 中存在未知标签')

    return pol


def build_unitcell_pol_df(frame_path: str, cubic_coord_path: str):
    atom_coord = read_data(frame_path)
    cubic_coord = read_data(cubic_coord_path)

    x_min, x_max = min(atom_coord[:, 1]), max(atom_coord[:, 1])
    y_min, y_max = min(atom_coord[:, 2]), max(atom_coord[:, 2])
    z_min, z_max = min(atom_coord[:, 3]), max(atom_coord[:, 3])
    x_avg = (x_max - x_min) / 40
    y_avg = (y_max - y_min) / 20
    z_avg = (z_max - z_min) / 20
    volume = x_avg * y_avg * z_avg

    # logger.info(f'atom_coord: \n{atom_coord[:, [1, 2, 3][:5]]}')
    kdtree = KDTree(atom_coord[:, [1, 2, 3]])

    df = pd.DataFrame(columns=['x', 'y', 'z', 'pol'])

    for raw in cubic_coord:
        # raw = cubic_coord[9]
        A_cubic = process_A_coord(raw)
        Ti_cubic = raw
        O_cubic = process_O_coord(raw)
        _, A_index = kdtree.query(A_cubic)
        _, Ti_index = kdtree.query(Ti_cubic)
        _, O_index = kdtree.query(O_cubic)
        A_coord = atom_coord[A_index]
        Ti_coord = atom_coord[Ti_index]
        O_coord = atom_coord[O_index]
        # logger.info(f'A_coord: \n{A_coord}')
        # logger.info(f'Ti_coord: \n{Ti_coord}')
        # logger.info(f'O_coord: \n{O_coord}')

        pol = cal_polarization(A_coord, Ti_coord, O_coord,
                               A_cubic, Ti_cubic, O_cubic, volume, atom_coord, kdtree)
        df.loc[len(df)] = [raw[0], raw[1], raw[2], pol]

        # break
    return df


if __name__ == "__main__":
    frame_path = "./data/timesteps_200k/frame_50000.npy"
    cubic_path = './data/cubic_402020/plane_all.npy'
    df = build_unitcell_pol_df(frame_path, cubic_path)

In [16]:
df

,x,y,z,pol
0,3.9127,3.9127,3.9127,"[0.010299937037717841, -0.003757462314622526, ..."
1,3.9127,3.9127,7.8254,"[0.00828515589456933, 0.003651860288164046, -0..."
2,3.9127,3.9127,11.7381,"[0.007832350045892193, -0.006920337414141828, ..."
3,3.9127,3.9127,15.6508,"[0.0006833867887721951, 0.00018434490194451871..."
4,3.9127,3.9127,19.5635,"[-0.010365948347561488, 0.0060006617390690195,..."
...,...,...,...,...
12307,148.6826,70.4286,54.7778,"[-0.0025718147590934274, 0.006909021117859855,..."
12308,148.6826,70.4286,58.6905,"[0.0024693813960306255, -0.010589905726619853,..."
12309,148.6826,70.4286,62.6032,"[-0.00016878614535005435, -0.00871956575239644..."
12310,148.6826,70.4286,66.5159,"[0.003888835467117773, 0.0010244842802704297, ..."


In [24]:
tmp = df['pol'].apply(lambda x: np.array(x) if isinstance(x, (list, np.ndarray)) else x)
pol_array = np.array(tmp.tolist(), dtype=np.float64)  # 将列表转换为 NumPy 数组
pol_array


array([[ 0.01029994, -0.00375746, -0.02827764],
       [ 0.00828516,  0.00365186, -0.04245233],
       [ 0.00783235, -0.00692034, -0.04509103],
       ...,
       [-0.00016879, -0.00871957, -0.00103912],
       [ 0.00388884,  0.00102448, -0.00308746],
       [-0.00919931, -0.00663892, -0.00598747]])

In [3]:
from scipy.spatial import KDTree

# 定义二维坐标点数据
points = [(2, 0, 3), (5, 6, 7), (8, 7, 1), (4, 3, 5)]

# 构建k-d树
kdtree = KDTree(points)

# 查询最近的坐标点
agv_position = (8, 6, 4)
nearest_distance, nearest_index = kdtree.query(agv_position, )
nearest_point = points[nearest_index]

print("最近的坐标点：", nearest_point)

最近的坐标点： (8, 7, 1)


In [ ]:
def bucket_sort_3d(points, n_buckets=10):
    if not points:
        return points

    # 找到每一维的最小值和最大值
    min_x, max_x = min(p[0] for p in points), max(p[0] for p in points)
    min_y, max_y = min(p[1] for p in points), max(p[1] for p in points)
    min_z, max_z = min(p[2] for p in points), max(p[2] for p in points)

    # 处理所有值相同的情况，避免除以零
    if max_x == min_x:
        max_x += 1e-10
    if max_y == min_y:
        max_y += 1e-10
    if max_z == min_z:
        max_z += 1e-10

    # 桶排序函数
    def bucket_sort_dim(arr, dim, min_val, max_val, n_buckets):
        buckets = [[] for _ in range(n_buckets + 1)]
        range_val = max_val - min_val

        # 分配到桶中
        for point in arr:
            index = int((point[dim] - min_val) / range_val * n_buckets)
            if index == n_buckets:  # 处理边界情况
                index -= 1
            buckets[index].append(point)

        # 对每个桶内的元素按当前维度排序
        for bucket in buckets:
            bucket.sort(key=lambda p: p[dim])

        # 合并结果
        return [p for bucket in buckets for p in bucket]

    # 依次按z、y、x排序（后排序的维度优先级更高）
    result = bucket_sort_dim(points, 2, min_z, max_z, n_buckets)  # 先按z排序
    result = bucket_sort_dim(result, 1, min_y, max_y, n_buckets)  # 再按y排序
    result = bucket_sort_dim(result, 0, min_x, max_x, n_buckets)  # 最后按x排序

    return result